## 先尋找職業所需的工作技能

## 先爬取外層頁

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import jieba
from jieba.analyse import extract_tags
from collections import Counter

In [2]:
#取得爬取網頁資訊所需的form data
payload = """find_key1:{}
strrec:
search_key_word:{}
search_type:job
us_menu:
search_item:1
search_from:index""".format("程式設計師","程式設計師")  #format內容為要查詢的職業

In [3]:
#使用一個function將form data轉換成dict的key-value pair才可放到requests的query中
def str2dict(data_str):
    data = {}
    for row in data_str.split('\n'):
        kv_list = row.split(":")
        data[kv_list[0]] = kv_list[1]
    return data

In [4]:
#測試一下看form data是否正確轉換成dict的key-value pair
data = str2dict(payload)
data

{'find_key1': '程式設計師',
 'search_from': 'index',
 'search_item': '1',
 'search_key_word': '程式設計師',
 'search_type': 'job',
 'strrec': '',
 'us_menu': ''}

In [5]:
def total_pages(html,formdata):
    formdata = str2dict(formdata)
    res = requests.post(html,data = formdata)
    res.encoding = 'utf8'
    soup = BeautifulSoup(res.text,'lxml')
    td_tag = soup.select_one('#top_title').find(lambda x: x.get('align') == 'right')
    pages = int(re.findall('\d+',td_tag.text)[2])
    return pages

In [6]:
all_pages = total_pages("https://www.yes123.com.tw/admin/job_refer_list.asp",payload)

In [7]:
# Pagination

res = requests.post("https://www.yes123.com.tw/admin/job_refer_list.asp",data=data)
res.encoding = 'utf8'
soup = BeautifulSoup(res.text,'lxml')
td_tag = soup.select_one("#top_title").find(lambda x: x.get('align') == 'right')
td_tag

<td align="right">
                            407 筆，
							第1 / 21頁
					</td>

In [8]:
total_records = int(re.findall('\d+',td_tag.text)[0])
total_records

407

In [9]:
#算出總頁數
total_page = int(total_records / 20) + 1
total_page

21

In [10]:
#算出最後一頁有幾筆
last_page = int(total_records % 20)
last_page

7

In [23]:
Links = []
for i in range(0,all_pages):
    data['strrec'] = i*20
    # data = str2dict(payload)
    # data

    res = requests.post("https://www.yes123.com.tw/admin/job_refer_list.asp",data=data)
    res.encoding = 'utf8'
    #     res.text

    soup = BeautifulSoup(res.text,'lxml')
        # 取得所有工作職缺
    titles =[job['title'] for job in soup.select('a.jobname')]


        # 取得所有工作職缺的連結
    urls = [job['href'] for job in soup.select('a.jobname')]
    uri = "https://www.yes123.com.tw/admin/"
    for links in urls:
        Links.append(uri+links)
#     job_list = []
#     for title,url in zip(titles, urls):
#         job_dict = {}
#         job_dict['title']  = title
#         job_dict['url'] = uri + url
#         job_list.append(job_dict)
#     Links.append(job_list)
# Links
#         Links.append(job_list)


In [20]:
len(Links)

407

In [30]:
cnt = Counter()
for Link in Links:
    try:
        all_links = Link
        url2 = all_links
        res2 = requests.get(url2)
        soup = BeautifulSoup(res2.text, 'lxml')
           

        for pcskill in soup.select('.comp_detail > ul > li'):

            if re.findall('電腦技能|工作技能',pcskill.text) and not re.findall('\d+\.',pcskill.text) :
                skill =pcskill.text.replace('電腦技能','').replace(' 未填寫','').replace('辦公室應用','').replace('、',',').replace('工作技能','').replace('網頁技術','').replace('程式設計','').replace('資料庫','').replace('：',',').replace('/',',').replace('／',',').replace(',,',',').strip()
                skill = skill.lower()
#                 print(skill.split(','))

                for skills in skill.split(','):
                    if re.findall('\w+',skills):

                        if skills in cnt:
                            cnt[skills] += 1
                        else:
                            cnt[skills] = 1
#                             print (skills)
             
    except:
        pass

print (cnt.most_common(200)) 
                    

[('ms sql', 78), ('html', 69), ('java', 62), ('asp.net', 54), ('word', 52), ('excel', 52), ('powerpoint', 50), ('oracle', 49), ('mysql', 48), ('php', 43), ('java script', 41), ('c#', 25), ('jquery', 25), ('c#.net', 24), ('linux', 24), ('asp', 22), ('outlook', 22), ('vb', 21), ('ajax', 20), ('jsp', 19), ('普通小型車', 17), ('c++', 17), (' 輕型機車', 16), ('windows', 14), ('pl', 14), ('css', 13), ('sql', 13), ('dhtml', 12), ('visual c++', 12), ('vb .net', 11), ('dreamweaver', 10), ('android', 10), ('ios', 10), ('c', 9), ('delphi', 9), ('web master', 8), ('developer', 8), ('j2ee', 8), ('photoshop', 8), (' 普通重型機車', 7), ('xml web services', 7), (' 軟體', 7), ('ip', 7), ('spring', 6), ('unix', 6), ('visual studio .net', 6), ('visual studio', 6), ('c++作業系統', 6), ('軟體工程系統開發', 6), ('軟體', 6), ('網路', 6), ('作業系統', 6), ('jquery作業系統', 6), ('普通重型機車', 5), ('javascript', 5), ('系統整合分析', 5), ('軟體應用', 5), ('access繪圖軟體', 5), ('xml', 5), ('tcp', 5), ('php作業系統', 5), ('outlook作業系統', 4), ('windows nt', 4), ('java作業系統', 4

In [31]:
import json

In [32]:
json.dump(cnt,open("Count.json","w"),ensure_ascii=False)